In [1]:
import numpy as np
from datasets import load_dataset
import pickle
import tensorflow as tf
from scipy import sparse
import scipy.sparse.linalg as sln
from tqdm import tqdm

In [2]:
from num2words import num2words
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
stop_words = set(nltk.corpus.stopwords.words('english'))
nltk.download('omw-1.4')
import re

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


### Przygotuj duży (>1000 elementów) zbiór dokumentów tekstowych w języku an-gielskim (np. wybrany korpus tekstów, podzbiór artykułów Wikipedii, zbiór doku-mentówHTMLuzyskanych za pomocąWeb crawlera, zbiór rozdziałów wyciętych zróżnych książek)

Wybrany zbiór składa się z dumpa wikipedii simple. Cały set składa się z 205328 stron, jednak ze względu na ograniczoną moc obliczeniową został ograniczony do 20000 w podstawowej wersji i 2000 dla wersji z usuwaniem szumu

In [3]:
dataset_dict = load_dataset("wikipedia", "20220301.simple")
dataset = dataset_dict['train']

Reusing dataset wikipedia (C:\Users\User\.cache\huggingface\datasets\wikipedia\20220301.simple\2.0.0\aa542ed919df55cc5d3347f42dd4521d05ca68751f50dbc32bae2a7f1e167559)


  0%|          | 0/1 [00:00<?, ?it/s]

In [4]:
dataset

Dataset({
    features: ['id', 'url', 'title', 'text'],
    num_rows: 205328
})

In [5]:
DOCUMENT_COUNT = dataset.num_rows

Funkcja używana do preprocessingu tekstów. Usuwanie "stop words", zamienie liter na małe, itd.

In [6]:
lemma = nltk.WordNetLemmatizer()
def preprocess(dicti):
    text = dicti['text']
    text = text.lower()
    text = text.replace("_","")
    text = re.sub(r'[^\w\s]','',text)
    text = re.sub('  +',' ',text)

    text_list = text.split()
    text = ""

    for word in text_list:
        word = lemma.lemmatize(word, pos="v")
        try: word = num2words(int(word))
        except ValueError: pass
        if word in stop_words: continue

        text += word + " "
    text = re.sub(r'[0-9]','',text)
    text = re.sub(r'[^\w\s]','',text)
    dicti['processed'] = text
    return dicti


In [7]:
dataset = dataset.map(lambda e: preprocess(e))
with open("dataset", "wb") as file:
    pickle.dump(dataset, file)

  0%|          | 0/205328 [00:00<?, ?ex/s]

In [8]:
with open("dataset", "rb") as d:
    dataset = pickle.load(d)

DOCUMENT_COUNT = 2_000

In [9]:
dataset.features

{'id': Value(dtype='string', id=None),
 'url': Value(dtype='string', id=None),
 'title': Value(dtype='string', id=None),
 'text': Value(dtype='string', id=None),
 'processed': Value(dtype='string', id=None)}

### Określ  słownik  słów  kluczowych  (termów)  potrzebny  do  wyznaczenia  wektorówcechbag-of-words(indeksacja). Przykładowo zbiorem takim może być unia wszyst-kich słów występujących we wszystkich tekstach.

Do tokenizacji został Tokenizer z biblioteki Tensorflow

In [10]:
tokenizer = tf.keras.preprocessing.text.Tokenizer()

In [11]:
tokenizer.fit_on_texts([dataset[i]['processed'] for i in tqdm(range(DOCUMENT_COUNT))])

100%|██████████| 2000/2000 [00:00<00:00, 11266.09it/s]


In [12]:
with open("tokenizer_2K", "wb") as file:
    pickle.dump(tokenizer, file)

In [13]:
len(tokenizer.word_counts)

54672

In [14]:
with open("tokenizer_2K", "rb") as d:
    tokenizer = pickle.load(d)

 Dla każdego dokumentujwyznacz wektor cechbag-of-words $d_j$ zawierający częstości występowania poszczególnych słów (termów) w tekście.
 Zbuduj rzadką macierz wektorów cechterm-by-document matrixw której wekto-ry cech ułożone  są kolumnowo $A_{m × n}= [d_1|d_2|...|d_n]$ (m jest liczbą  termów  wsłowniku, anliczbą dokumentów)
 w moim przypadku wektory układane są rzędami, a nie kolumnami. Jest to brane pod uwagę w kolejnych krokach

In [15]:
BATCH = 400
TBD_matrix = sparse.csr_matrix(tokenizer.texts_to_matrix([dataset[i]['processed'] for i in range(DOCUMENT_COUNT//BATCH)], mode="count"), dtype=np.float64)


for i in tqdm(range(1, BATCH)):
    matrix = sparse.csr_matrix(tokenizer.texts_to_matrix([dataset[i]['processed']
              for i in range((DOCUMENT_COUNT//BATCH)*i, (DOCUMENT_COUNT//BATCH)*(i+1) )], mode="count"), dtype=np.float64)
    TBD_matrix = sparse.vstack([TBD_matrix, matrix])



100%|██████████| 399/399 [00:02<00:00, 136.40it/s]


In [16]:
with open("vTBD_matrix_2K", "wb") as file:
    pickle.dump(TBD_matrix, file)

Przetwórz wstępnie otrzymany zbiór danych mnożąc elementybag-of-wordsprzezinverse document frequency. Operacja ta pozwoli na redukcję znaczenia często wy-stępujących słów. $IDF(w) = \frac {logN} {n_w}$, gdzie $ n_w $ jest liczbą dokumentów, w których występuje słowow, aNjest całkowitąliczbą dokumentów.

In [17]:
IDF = np.zeros(shape=(TBD_matrix.shape[1]))
for idx, w in tokenizer.index_word.items():
    IDF[idx] = np.log(DOCUMENT_COUNT / tokenizer.word_docs[w])

In [18]:
TBD_matrix = TBD_matrix.multiply(IDF).tocsr()
with open("vTBD_matrix_IDF_2K", "wb") as d:
    pickle.dump(TBD_matrix, d)

In [19]:
with open("vTBD_matrix_IDF_2K", "rb") as d:
    TBD_matrix = pickle.load(d)

Napisz  program  pozwalający  na  wprowadzenie  zapytania  (w  postaci  sekwencjisłów)  przekształcanego  następnie  do  reprezentacji  wektorowejq(bag-of-words).Program ma zwrócić k dokumentów najbardziej zbliżonych do podanego zapytania. Użyj korelacji między wektorami jako miary podobieństwa
$$ cosθ_j = \frac {q^Td_j} {‖q‖‖d_j‖} = \frac {q^TAe_j}{‖q‖‖Ae_j‖} $$

In [20]:
def similarity_metric_1(q, matrix, i):
    return (matrix.getrow(i).dot(q[0]) / sln.norm(matrix.getrow(i)))[0]

In [21]:
def search(query, matrix, metric , k=10):
    cos = []
    q = tokenizer.texts_to_matrix([query], mode="binary")
    q_norm = np.linalg.norm(q[0])
    q = q / q_norm

    for i in tqdm(range(DOCUMENT_COUNT)):
        cos.append(metric(q, matrix, i))

    # top_k = np.argpartition(cos, -k)[-k:]
    top_k = np.argsort(cos)[-k:][::-1]
    for i in top_k:
        print("Title: {:<35} close: {:<20} url: {:<34}".format(dataset[int(i)]['title'], cos[int(i)], dataset[int(i)]['url']))

In [22]:
search("roman", TBD_matrix, similarity_metric_1)

100%|██████████| 2000/2000 [00:00<00:00, 4284.93it/s]

Title: Roman                               close: 0.5808212199281281   url: https://simple.wikipedia.org/wiki/Roman
Title: Roman Empire                        close: 0.2954126184236678   url: https://simple.wikipedia.org/wiki/Roman%20Empire
Title: Holy Roman Empire                   close: 0.2523634709164451   url: https://simple.wikipedia.org/wiki/Holy%20Roman%20Empire
Title: 9                                   close: 0.24050016434573168  url: https://simple.wikipedia.org/wiki/9
Title: 8                                   close: 0.20692046254854818  url: https://simple.wikipedia.org/wiki/8
Title: Saint David                         close: 0.19168945637541088  url: https://simple.wikipedia.org/wiki/Saint%20David
Title: 7                                   close: 0.18462009873706187  url: https://simple.wikipedia.org/wiki/7
Title: Venus (mythology)                   close: 0.17636578571505107  url: https://simple.wikipedia.org/wiki/Venus%20%28mythology%29
Title: 12                        

 Zastosuj normalizację wektorów cechdji wektoraq, tak aby miały one długość 1. Użyj zmodyfikowanej miary podobieństwa otrzymując $|q^TA|= [|cosθ_1|,|cosθ_2|,...,|cosθ_n|] $

In [23]:
from sklearn.preprocessing import normalize
normalize(TBD_matrix, norm='l1', axis=1, copy=False)

<2000x54673 sparse matrix of type '<class 'numpy.float64'>'
	with 393827 stored elements in Compressed Sparse Row format>

In [24]:
TBD_matrix.getrow(0).sum()

0.9999999999999971

In [25]:
with open("vTBD_normalized", "rb") as d:
    TBD_matrix = pickle.load(d)

In [26]:
with open("vTBD_normalized_2K", "wb") as d:
    pickle.dump(TBD_matrix, d)

W  celu  usunięcia  szumu  z  macierzyAzastosuj  SVD  i $low$ $rank$ $approximation $ otrzymując
$$ A \approx A_k = U_kD_kV^T_k= [u_1 | \hdots | u_k]\begin{bmatrix} σ_1 & & \\ & \ddots & \\ & &σ_k
 \end{bmatrix} \begin{bmatrix}v^T_1\\ \vdots \\ v^T_k \end{bmatrix}=\sum_{i=1}^{k} σ_iu_iv_i^T $$
oraz nową miarę podobieństwa
$$cosθ_j = \frac {q^TA_ke_j}{‖q‖‖A_ke_j‖}$$

In [27]:
import scipy.sparse.linalg as ssl
u, s, vt = ssl.svds(TBD_matrix, k=1000)

In [28]:
u, s, vt  = sparse.csr_matrix(u, dtype=np.float32), sparse.diags(s) , sparse.csr_matrix(vt, dtype=np.float32)

In [29]:
with open("tuple3_2K", "wb") as d:
    pickle.dump((u, s, vt), d)

In [ ]:
low_rank = u @ s @ vt

In [ ]:
with open("low_rank", "wb") as d:
    pickle.dump(low_rank, d)

In [ ]:
def similarity_metric_2(q, matrix, i):
    return (q[0].transpose() @ matrix).getrow(i)

In [ ]:
with open("low_rank", "rb") as d:
    low_rank = pickle.load(d)

In [ ]:
search("tank", low_rank, similarity_metric_1)

In [ ]:
with open("vTBD_matrix_2K", "rb") as d:
    m = pickle.load(d)
search("april month", m, similarity_metric_1)

Po usunięciu szumu z macierzy (k = 1000) wyniki wydają się nieco lepsze. W wynikach bez aproksymacji w wynikach pojawiają się mało sensowne pozycje (1992, 2005, 1603). W wersji z usuwaniem dalej się one pojawiają, ale są gorzej oceniane.

In [ ]:
search("april month", TBD_matrix, similarity_metric_1)

Bez wykorzystania IDF wyniki są znacznie słabsze jakościowo. IDF obniża wartość popularnych słów, co sprawia, że słowa kluczowe mają większy wpływ na ostateczny wynik wyszukiwania.